In [40]:
pip install CSV

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement CSV (from versions: none)
ERROR: No matching distribution found for CSV


In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow import keras
import torch
from keras.preprocessing.image import ImageDataGenerator
from torch.utils.data import Dataset
from torchvision import transforms,datasets 
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
import random
from tensorflow.keras.applications import ResNet50
import csv
from PIL import Image
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.optimizers import Adam

In [20]:
class MultiLabelImageDataset(keras.utils.Sequence):
    def __init__(self, image_dir, label_file, num_classes, tokenizer, max_length, batch_size=32, shuffle=True, subset='train'):
        self.image_dir = image_dir
        self.num_classes = num_classes
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.data = []
        
        # Load labels and captions from the CSV file
        with open(label_file, 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                image_id, labels_str, caption = row
                labels = [int(label) for label in labels_str.split()]
                self.data.append((os.path.join(image_dir, f"{image_id}.jpg"), labels, caption))
        
        # Split the data into train and test sets
        random.shuffle(self.data)
        split_idx = int(len(self.data) * 0.8)  # 80% for training, 20% for testing
        if self.subset == 'train':
            self.data = self.data[:split_idx]
        else:
            self.data = self.data[split_idx:]
        
        self.indexes = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = []
        batch_text = []
        batch_targets = []
        for i in batch_indexes:
            image_path, labels, caption = self.data[i]

            # Load and preprocess the image
            img = Image.open(image_path)
            img = img.resize((224, 224))
            img = np.array(img)
            img = keras.applications.resnet50.preprocess_input(img)
            batch_images.append(img)

            # Tokenize and encode the caption
            encoded_caption = self.tokenizer.encode_plus(
                caption,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_token_type_ids=False,
                return_tensors='tf'
            )
            batch_text.append(encoded_caption['input_ids'][0])  # Extract the input IDs from the encoded caption

            # Convert labels to a one-hot encoded numpy array
            target = np.zeros(self.num_classes)
            for label in labels:
                target[label - 1] = 1
            batch_targets.append(target)

        batch_images = np.array(batch_images)
        batch_text = np.array(batch_text)
        batch_targets = np.array(batch_targets)

        return (batch_images, batch_text), batch_targets  # Return a tuple of inputs and targets
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [21]:


# Initialize the BERT tokenizer
tokenizer_ = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128  # Maximum sequence length for the text input

train_dataset = MultiLabelImageDataset(label_file='train.csv', image_dir = 'data/', num_classes = 18, tokenizer=tokenizer_ , max_length = 128, batch_size=32, subset='train')
test_dataset = MultiLabelImageDataset(label_file='train.csv', image_dir = 'data/', num_classes = 18, tokenizer=tokenizer_ , max_length = 128, batch_size=32, subset='test')




In [151]:
 # Load ResNet50 model with pre-trained weights
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

 # Load BERTmini model from pre-trained weights
bertmini_model = TFBertModel.from_pretrained('google/bert_uncased_L-4_H-256_A-4', from_pt=True)
    
 # Load TinyBERT model
tinybert_model = TFBertModel.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.self.key.bias', 'bert.pooler.dense.weight', 'bert.encoder.layer.2.output.dense.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.intermediate.dense.weight', 'bert.encoder.layer.1.output.LayerNorm.weight', 'cls.seq_relationship.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.self.key.weight', 'bert.pooler.dense.bias', 'bert.encoder.layer.3.intermediate.dense.bias', 'bert.encoder.layer.0.attent

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.self.key.bias', 'bert.pooler.dense.weight', 'bert.encoder.layer.2.output.dense.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'fit_denses.0.weight', 'bert.encoder.layer.2.output.dense.weight', 'bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.intermediate.dense.weight', 'bert.encoder.layer.1.output.LayerNorm.weight', 'cls.seq_relationship.weight', 'fit_denses.4.weight', 'fit_denses.3.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.self.key.weight', 'fit_denses.2.weight', 'bert.pool

In [152]:
# Freeze the ResNet50 and TinyBERT layers
#for layer in resnet_model.layers:
#    layer.trainable = False
#for layer in tinybert_model.layers:
#    layer.trainable = False

# Fine-tune the ResNet50 and TinyBERT models
for layer in resnet_model.layers[:]:
    layer.trainable = False
for layer in tinybert_model.layers[:]:
    layer.trainable = False
for layer in bertmini_model.layers[:]:
    layer.trainable = False

In [159]:
# Create the image input
image_input = keras.Input(shape=(224, 224, 3))
image_features = resnet_model(image_input)
image_features = keras.layers.GlobalAveragePooling2D()(image_features)

In [160]:
# Create the text input
text_input = keras.Input(shape=(max_length,), dtype=tf.int32)
text_features = bertmini_model(text_input)[1]  # Use the pooled output

In [161]:
# Concatenate the image and text features
combined_features = keras.layers.concatenate([image_features, text_features])

In [162]:
outputs = keras.layers.Dense(18, activation='sigmoid')(combined_features) # num_classes is 18

In [163]:
# Create the final model
model = keras.models.Model(inputs=[image_input, text_input], outputs=outputs)

In [164]:
# Compile the model with a custom learning rate
learning_rate = 0.00002
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
epochs = 1
model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        workers=1,
        use_multiprocessing=False
    )

750/750 [==============================] - ETA: 0s - loss: 0.1225 - accuracy: 0.7967

In [15]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

188/188 [==============================] - 340s 2s/step - loss: 0.0821 - accuracy: 0.8200
Test Loss: 0.0821
Test Accuracy: 0.8200
